In [3]:
import fitz
from collections import defaultdict
from extract_content import extract_content
import json
import math, re
import string
import pprint


In [2]:
# read pdf
filename = 'taiwan-11-ebook.pdf'
with fitz.open(filename) as doc:
    pages = [page.getText('dict') for page in doc.pages()]
    pages = []
    for page in doc.pages():
        cont = page.getText('dict')
        pages.append(cont)

In [4]:
text_content, image_content = extract_content(pages)

 [ INFO ] start to extract content. #page = 419 
 [ INFO ] processed finished 


In [5]:
test1 = []
for i in range(61,63):
    test1.append(text_content[i])
    

In [5]:
# To Make a Dictionary with LonelyPlanet from Page 61 to Page 341 
Dict = {}
Heading = []
for i in range(61,341):
    for block in text_content[i]:
        lines = block['lines']
        for line in lines:
            spans = line['spans']
            for span in spans:
                if span['size'] == 9.0:
                    title = span['text']
                    # if span['text'] not in Heading:
                    if title not in Heading:
                        Heading.append(span['text'])
                        #print(title)
                        Dict[title] =''
                elif span['size'] == 7.800000190734863:
                    english_only = ''.join(x for x in span['text'] if ord(x) < 256)
                    # englisch_only = list(english_only)
                    Dict[title] += english_only
                    #pass
with open('lonelyplanet_data.json', 'w') as outfile:
    json.dump(Dict, outfile)

In [6]:
with open('lonelyplanet_data.json', 'r') as f:
    content = json.load(f)

In [7]:
#import re
#import json
import spacy
nlp = spacy.load("en_core_web_sm")

def words(text): return re.findall("([a-zA-Z’-]+|[0-9]+)", text)
def ngrams(tokens, n): return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n) ]

LB_Dict = {}
count_lb = defaultdict(lambda: 0)
content = json.load(open('lonelyplanet_data.json', 'r'))

for key, value in content.items(): 
    for sentence in sent_detector.tokenize(content[key]):
        LB_Dict[key] = []
        for i in [3,4,5]:
            for ngram in ngrams(words(sentence),i):
                doc = nlp(ngram)
                for token in doc[0:1]:
                    if token.pos_ != 'AUX' and token.tag_ in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                        # print(ngram)
                        count_lb[ngram] += 1
                    elif token.pos_ == 'NOUN':
                        count_lb[ngram] += 1
                    elif token.pos_ == 'ADJ':
                        count_lb[ngram] += 1
        lex_bundles = [ (ngram, count) for ngram, count in count_lb.items() if count>=3 ]
        for ngram, count in sorted(lex_bundles, key=lambda x: -x[1]):
            # print ('\t'.join([ngram, str(count)]))
            LB_Dict[key].append((ngram, str(count)))

In [8]:
with open('LB_lonelyplanet.json', 'w') as outfile:
    json.dump(LB_Dict, outfile)